In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from album import album
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-23 06:09:19.827022


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
alb  = album()
albs = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Collections

In [35]:
%load_ext autoreload
%autoreload
col = collections(disc)
#col.parseCollections(force=False)
#col.createCollectionDBs()
#col.parseCollectionsForAlbums(force=True)
#from webUtils import getHTML
#bsdata = getHTML('/Volumes/Music/Discog/collections/Rock & Roll-US-2010-1.p')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
x = '/Volumes/Music/Discog/diagnostic/albumKnownIDs.p'
saveFile(idata={}, ifile=x)
#disc.getDiagnosticAlbumIDs()

  --> This file is 14B.


In [ ]:
col.createCollectionDBsForAlbums()

# Download Searched Artists

In [5]:
arts.searchDiscogForArtist("Dodo And The Dodo's")



===================== Searching For Dodo And The Dodo's =====================
Downloading: https://www.discogs.com/search/?q=Dodo%20And%20The%20Dodo%27s&limit=250&type=artist
Found 42 artists
2 / 42 	: 6 	 https://www.discogs.com/artist/638330-Dodo-And-The-Dodos?sort=year%2Casc&limit=500
3 / 42 	: 6 	 https://www.discogs.com/artist/314317-Dodo-3?sort=year%2Casc&limit=500
  --> This file is 16.6kB.
4 / 42 	: 6 	 https://www.discogs.com/artist/416350-Dodo-Nkishi?sort=year%2Casc&limit=500
  --> This file is 17.0kB.
5 / 42 	: 6 	 https://www.discogs.com/artist/869788-Dodo-Gad?sort=year%2Casc&limit=500
6 / 42 	: 7 	 https://www.discogs.com/artist/1977712-Joe-Dodo-And-The-Groovers?sort=year%2Casc&limit=500
7 / 42 	: 6 	 https://www.discogs.com/artist/467756-Dodo-Greene?sort=year%2Casc&limit=500
8 / 42 	: 7 	 https://www.discogs.com/artist/5436454-Dodo-Ben-Radis?sort=year%2Casc&limit=500
  --> This file is 16.1kB.
9 / 42 	: 7 	 https://www.discogs.com/artist/4612784-Armandinho-E-O-Trio-El%C

HTTPError: HTTP Error 429: Too Many Requests

# Parse Artists (Parallel)

In [6]:
arts.parseArtistModValFiles(70, force=True)
for modVal in range(71,75):
    parseArtists(modVal)

Parsing Artist Files For ModVal 70
Forcing Reloads of ModVal=70
  Processing 4762 files.
	Processed 0/4762 files.
	Processed 500/4762 files.
	Processed 1000/4762 files.
	Processed 1500/4762 files.
	Processed 2000/4762 files.
	Processed 2500/4762 files.
	Processed 3000/4762 files.
	Processed 3500/4762 files.
	Processed 4000/4762 files.
	Processed 4500/4762 files.
Saving 4762 new artist IDs to /Volumes/Music/Discog/artists-db/70-DB.p
  --> This file is 9.6MB.
Parsing Artist Files For ModVal 71
Saving 63 new artist IDs to /Volumes/Music/Discog/artists-db/71-DB.p
  --> This file is 9.7MB.
Parsing Artist Files For ModVal 72
Saving 43 new artist IDs to /Volumes/Music/Discog/artists-db/72-DB.p
  --> This file is 10.0MB.
Parsing Artist Files For ModVal 73
Saving 56 new artist IDs to /Volumes/Music/Discog/artists-db/73-DB.p
  --> This file is 10.2MB.
Parsing Artist Files For ModVal 74
Saving 59 new artist IDs to /Volumes/Music/Discog/artists-db/74-DB.p
  --> This file is 10.9MB.


In [3]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProcs=4, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel()
elapsed(start2, cmt2)
#arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Sep 14, 2019 23:07:45 for Parsing All Artists
Current Time is Sat Sep 14, 2019 23:07:45 for Parallel Artist Parsing
Current Time is Sat Sep 14, 2019 23:07:45 for Done with Parallel Artist Parsing
Process [{0}] took 179 microseconds.
Current Time is Sat Sep 14, 2019 23:07:45 for Done with Parsing All Artists
Process [{0}] took 751 microseconds.


In [ ]:
from collections import Counter
albcnts = disc.getCollectionAlbumRefCountsData()
albarts = disc.getCollectionAlbumRefArtistsData()
albrefs = disc.getAlbumRefToIDData()
alberrs = getFile('alberrs.p')

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

In [ ]:
def getAlbums(todo=False):
    retval = {}
    n = len(albcnts)
    for i,item in enumerate(albcnts.most_common()):
        ref, cnt = item[0], item[1]
        if albrefs.get(ref) is not None:
            continue
        aas = albarts[ref]

        url=albs.getAlbumURL(ref)
        if alberrs.get(url):
            continue

        for aa in aas:
            artistID   = dutils.getArtistID(aa)
            if artistID is None:
                artistID = "NAN"
                continue
            else:
                pass

            albumID  = dutils.getAlbumID(ref)
            savename = albs.getAlbumSavename(artistID, albumID)
            if isFile(savename):
                continue
                
            if todo:
                retval[url] = savename
                continue

            print("{0}/{1}\tGetting: {2}".format(i,n,savename))
            retval = albs.downloadAlbumURLData(url=url, savename=savename, artistID=artistID, sleeptime=1.75)
            if retval is False:
                alberrs[url] = True
                saveFile(idata=alberrs, ifile="alberrs.p")
                
    if todo:
        return retval

In [ ]:
retval = getAlbums(todo=False)

# Artists

In [13]:
for modVal in list(range(7,10)):
    parseArtists(modVal)

Parsing Artist Files For ModVal 7
Saving 22 new artist IDs to /Volumes/Music/Discog/artists-db/7-DB.p
  --> This file is 10.1MB.
Parsing Artist Files For ModVal 8
Saving 22 new artist IDs to /Volumes/Music/Discog/artists-db/8-DB.p
  --> This file is 9.8MB.
Parsing Artist Files For ModVal 9
Saving 17 new artist IDs to /Volumes/Music/Discog/artists-db/9-DB.p
  --> This file is 9.5MB.


# Parallel Artist Process

In [5]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProc=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
parseArtistsParallel()
elapsed(start2, cmt2)
arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Sat Sep 14, 2019 18:14:41 for Parsing All Artists
Current Time is Sat Sep 14, 2019 18:14:41 for Parallel Artist Parsing
Current Time is Sat Sep 14, 2019 18:14:41 for Done with Parallel Artist Parsing
Process [{0}] took 105 microseconds.
Current Time is Sat Sep 14, 2019 18:14:41 for Done with Parsing All Artists
Process [{0}] took 508 microseconds.


In [14]:
start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
elapsed(start2, cmt2)
arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

Current Time is Sat Sep 14, 2019 19:41:54 for Parsing All Artists
Current Time is Sat Sep 14, 2019 19:41:54 for Parallel Artist Parsing
Current Time is Sat Sep 14, 2019 19:41:54 for Done with Parallel Artist Parsing
Process [{0}] took 175 microseconds.
Current Time is Sat Sep 14, 2019 19:41:54 for Building Artist Metadata DB
Loading data from /Volumes/Music/Discog/db/ArtistNameToID.p
  --> This file is 9.5MB.
Loading /Volumes/Music/Discog/db/ArtistNameToID.p
Loading data from /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
  --> This file is 20.9MB.
Loading /Volumes/Music/Discog/db/ArtistVariationNameToIDs.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 9.1MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
Loading data from /Volumes/Music/Discog/db/ArtistRefToID.p
  --> This file is 11.3MB.
Loading /Volumes/Music/Discog/db/ArtistRefToID.p
Loading data from /Volumes/Music/Discog/db/ArtistIDToRef.p
  --> This file is 10.9MB.
Loading /Volumes/Music/

# Recent Artists

In [ ]:
from os import path
from datetime import datetime, timedelta

two_days_ago = datetime.now() - timedelta(days=2)

filetime = datetime.fromtimestamp(path.getctime(file_path))

if filetime < two_days_ago:
  print "File is more than two days old."

# Albums

In [ ]:
def parseAlbums(modVal, force=False):
    albs.parseAlbumModValFiles(modVal=modVal, force=force)
    
%load_ext autoreload
%autoreload
from album import album
from albums import albums
from multiprocessing import Pool
import time
albs = albums(disc)


def f_amp(modVals=100):
    nProcs = 3
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseAlbums, range(100))
 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    print("Runing .",end="")
    while not result.ready():
        #print(".",end="")
        time.sleep(10)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Albums")
start2,cmt2=clock("Parallel Album Parsing")
#albs.parseAlbumModValFiles(modVal='NAN', force=False)
f_amp()
elapsed(start2, cmt2)
albs.buildMetadata(force=False)
elapsed(start, cmt)

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
#albs = albums(disc)
#toget = albs.findMissingArtistsFromAlbums()

# Albums From Known Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=10, mediaTypes=None, rand=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value
Loading /Volumes/Music/Discog/artists-db/29-DB.p... Found 4767 Artists
1 / 4767 	: 0 	 1000529 	 Freddie Meyer
	Downloaded 0/5 entries of type Singles & EPs
  Found album code 7898722 with 1 artists
  Saving /Volumes/Musi

  Found album code 3811715 with 9 artists
  Saving /Volumes/Music/Discog/albums/3/20703/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/29/100729/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/34/237134/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/13/471313/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/56/426056/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/14/471314/3811715.p
  --> This file is 16.6kB.
  Saving /Volumes/Music/Discog/albums/42/230642/3811715.p
  --> This file is 16.6kB.
  Exists /Volumes/Music/Discog/albums/60/276060/3811715.p
  Saving /Volumes/Music/Discog/albums/NAN/NAN/3811715.p
  --> This file is 16.6kB.
  Done and sleeping...
  Found album code 13304580 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/100729/13304580.p
  --> This file is 15.8kB.
  Done and sleeping...

== Download Rate: 25 / 1.2 = 21.1

	Downloaded 0

  Found album code 8101276 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1009029/8101276.p
  --> This file is 15.7kB.
  Done and sleeping...
  Found album code 1009050 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1009029/1009050.p
  --> This file is 17.6kB.
  Done and sleeping...
  Found album code 887161 with 2 artists
  Saving /Volumes/Music/Discog/albums/99/4134799/887161.p
  --> This file is 15.8kB.
  Saving /Volumes/Music/Discog/albums/29/1009029/887161.p
  --> This file is 15.8kB.
  Done and sleeping...
  Found album code 1754347 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1009029/1754347.p
  --> This file is 17.7kB.
  Done and sleeping...
	Downloaded 0/3 entries of type Singles & EPs
  Found album code 1090184 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1009029/1090184.p
  --> This file is 14.4kB.
  Done and sleeping...
  Found album code 1204038 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1009029/1204038.p
  --> This file

  --> This file is 17.1kB.
  Done and sleeping...
	Downloaded 0/3 entries of type Miscellaneous
  Found album code 4852619 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/10129/4852619.p
  --> This file is 15.4kB.
  Done and sleeping...
  Found album code 378196 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/10129/378196.p
  --> This file is 14.7kB.
  Done and sleeping...
  Found album code 3824949 with 2 artists
  Saving /Volumes/Music/Discog/albums/24/137624/3824949.p
  --> This file is 14.4kB.
  Saving /Volumes/Music/Discog/albums/29/10129/3824949.p
  --> This file is 14.4kB.
  Done and sleeping...
16 / 4767 	: 17 	 1013029 	 Ian & Sylvia
	Downloaded 10/12 entries of type Albums
	Downloaded 0/8 entries of type Singles & EPs
  Found album code 873458 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1013029/873458.p
  --> This file is 14.9kB.
  Done and sleeping...
  Found album code 705544 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1013029/705544.

  Found album code 1027577 with 2 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/1027577.p
  --> This file is 16.0kB.
  Saving /Volumes/Music/Discog/albums/19/1900819/1027577.p
  --> This file is 16.0kB.
  Done and sleeping...
	Downloaded 0/9 entries of type Singles & EPs
  Found album code 6294747 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/6294747.p
  --> This file is 16.2kB.
  Done and sleeping...
  Found album code 5413705 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/5413705.p
  --> This file is 17.1kB.
  Done and sleeping...
  Found album code 1589865 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/1589865.p
  --> This file is 15.6kB.
  Done and sleeping...
  Found album code 7159698 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/7159698.p
  --> This file is 14.5kB.
  Done and sleeping...
  Found album code 2128544 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1016829/2128544.p
  --> This f

...

  Found album code 6754872 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062329/6754872.p
  --> This file is 16.8kB.
  Done and sleeping...
  Found album code 6830576 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062329/6830576.p
  --> This file is 18.8kB.
  Done and sleeping...
  Found album code 5039530 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062329/5039530.p
  --> This file is 17.0kB.
  Done and sleeping...
71 / 4767 	: 1 	 1062429 	 Sheryfa Luna
	Downloaded 1/5 entries of type Albums
  Found album code 992961 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062429/992961.p
  --> This file is 16.1kB.
  Done and sleeping...
  Found album code 13716366 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062429/13716366.p
  --> This file is 17.7kB.
  Done and sleeping...
  Found album code 1395324 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1062429/1395324.p
  --> This file is 16.3kB.
  Done and sleeping...
  Found album c

  Found album code 4671154 with 5 artists
  Saving /Volumes/Music/Discog/albums/80/4156680/4671154.p
  --> This file is 18.2kB.
  Saving /Volumes/Music/Discog/albums/29/1065329/4671154.p
  --> This file is 18.2kB.
  Saving /Volumes/Music/Discog/albums/11/2982811/4671154.p
  --> This file is 18.2kB.
  Saving /Volumes/Music/Discog/albums/87/2277687/4671154.p
  --> This file is 18.2kB.
  Saving /Volumes/Music/Discog/albums/91/1722791/4671154.p
  --> This file is 18.2kB.
  Done and sleeping...
  Found album code 10541607 with 3 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/10541607.p
  --> This file is 17.3kB.
  Saving /Volumes/Music/Discog/albums/83/55683/10541607.p
  --> This file is 17.3kB.
  Saving /Volumes/Music/Discog/albums/29/1065329/10541607.p
  --> This file is 17.3kB.
  Done and sleeping...
76 / 4767 	: 1 	 1066629 	 Candypants
	Downloaded 1/1 entries of type Albums
	Downloaded 0/1 entries of type Singles & EPs
  Found album code 2569541 with 1 artists
  Saving /Volumes/

  Found album code 9496373 with 5 artists
  Exists /Volumes/Music/Discog/albums/77/2363177/9496373.p
  Exists /Volumes/Music/Discog/albums/78/5430678/9496373.p
  Exists /Volumes/Music/Discog/albums/65/3582365/9496373.p
  Exists /Volumes/Music/Discog/albums/49/1403149/9496373.p
  Exists /Volumes/Music/Discog/albums/64/1070064/9496373.p
  Done and sleeping...
  Found album code 13097867 with 2 artists
  Exists /Volumes/Music/Discog/albums/25/192325/13097867.p
  Exists /Volumes/Music/Discog/albums/70/3966270/13097867.p
  Done and sleeping...
  Found album code 1267351 with 2 artists
  Exists /Volumes/Music/Discog/albums/25/192325/1267351.p
  Exists /Volumes/Music/Discog/albums/10/1070010/1267351.p
  Done and sleeping...
  Found album code 5823839 with 4 artists
  Exists /Volumes/Music/Discog/albums/42/95542/5823839.p
  Saving /Volumes/Music/Discog/albums/32/778532/5823839.p
  --> This file is 17.5kB.
  Saving /Volumes/Music/Discog/albums/80/2214580/5823839.p
  --> This file is 17.5kB.
  S

  Found album code 3560539 with 2 artists
  Saving /Volumes/Music/Discog/albums/29/1070629/3560539.p
  --> This file is 15.9kB.
  Saving /Volumes/Music/Discog/albums/28/1570628/3560539.p
  --> This file is 15.9kB.
  Done and sleeping...
  Found album code 2482838 with 1 artists
  Saving /Volumes/Music/Discog/albums/29/1070629/2482838.p
  --> This file is 16.7kB.
  Done and sleeping...
  Found album code 1319055 with 3 artists
  Saving /Volumes/Music/Discog/albums/29/1070629/1319055.p
  --> This file is 16.3kB.
  Saving /Volumes/Music/Discog/albums/2/214402/1319055.p
  --> This file is 16.3kB.
  Saving /Volumes/Music/Discog/albums/13/1089913/1319055.p
  --> This file is 16.3kB.
  Done and sleeping...
  Found album code 1319531 with 5 artists
  Saving /Volumes/Music/Discog/albums/29/1070629/1319531.p
  --> This file is 16.9kB.
  Saving /Volumes/Music/Discog/albums/61/144461/1319531.p
  --> This file is 16.9kB.
  Saving /Volumes/Music/Discog/albums/3/78003/1319531.p
  --> This file is 16.

	Downloaded 0/1 entries of type Technical
  Found album code 968523 with 1 artists
  Exists /Volumes/Music/Discog/albums/46/353446/968523.p
  Done and sleeping...
	Downloaded 0/6 entries of type Visual
  Found album code 5311161 with 1 artists
  Saving /Volumes/Music/Discog/albums/19/287519/5311161.p
  --> This file is 19.4kB.
  Done and sleeping...
  Found album code 9813170 with 1 artists
  Saving /Volumes/Music/Discog/albums/21/110921/9813170.p
  --> This file is 17.2kB.
  Done and sleeping...
  Found album code 7152396 with 1 artists
  Saving /Volumes/Music/Discog/albums/98/650898/7152396.p
  --> This file is 18.1kB.
  Done and sleeping...
  Found album code 271914 with 1 artists
  Exists /Volumes/Music/Discog/albums/29/1277429/271914.p
  Done and sleeping...
  Found album code 2319666 with 1 artists
  Saving /Volumes/Music/Discog/albums/92/1689692/2319666.p
  --> This file is 23.7kB.
  Done and sleeping...
  Found album code 586635 with 1 artists
  Exists /Volumes/Music/Discog/alb

In [ ]:
from searchUtils import findExt

In [ ]:
def getAlbumSavename(artistID, albumID):
    albumsDir   = disc.getAlbumsDir()
    artistIDDir = setDir(artistModDir, artistID)
    savename    = setFile(artistIDDir, "{0}.p".format(albumID))
    return savename

albumsDir = disc.getAlbumsDir()
artistModDir = setSubDir(albumsDir, str(modVal))
mediaTypes = ['Albums']
mediaTypes = None
maxAlbums  = 3

nArtists = len(dbdata)
iArtists = 0
#for artistID, artistData in dbdata.items():
if True:
    artistID = '1007579'
    artistData = dbdata[artistID]
    iArtists += 1
    
    artistIDDir = setDir(artistModDir, artistID)
    
    downloadedFiles = findExt(artistIDDir, ext=".p")
    print(iArtists,'/',nArtists,'\t:',len(downloadedFiles),'\t',artistData.ID.ID,'\t',artistData.artist.name)

    media = artistData.media.media
    for mediaType, mediaTypeData in media.items():
        if mediaTypes is not None:
            if mediaType not in mediaTypes:
                continue
                
        nget = 0        
        toGetMediaIDs = {mediaIDData.code: mediaIDData.url for mediaIDData in mediaTypeData}
        
        toGet = {}
        known = {}
        for albumID,albumURL in toGetMediaIDs.items():
            savename = getAlbumSavename(artistID,albumID)
            if not isFile(savename):
                toGet[albumURL] = savename
            else:
                known[albumURL] = savename

        nToGet = len(toGetMediaIDs)-len(toGet)
        print("\tDownloaded {0}/{1} entries of type {2}".format(nToGet, len(toGetMediaIDs), mediaType))
        for albumURL,savename in toGet.items():
            if len(known) >= maxAlbums:
                break               
            known[albumURL] = savename
        break

In [ ]:
toGet

# New Artists

In [49]:
arts.searchDiscogForArtist("Steve Nieve")



===================== Searching For Toe =====================
Downloading: https://www.discogs.com/search/?q=Toe&limit=250&type=artist
Found 251 artists
2 / 251 	: 7 	 https://www.discogs.com/artist/1333885-toe-5?sort=year%2Casc&limit=500
3 / 251 	: 7 	 https://www.discogs.com/artist/2638421-Toe-7?sort=year%2Casc&limit=500
4 / 251 	: 6 	 https://www.discogs.com/artist/910141-Toe-To-Toe?sort=year%2Casc&limit=500
5 / 251 	: 6 	 https://www.discogs.com/artist/225382-Tic-Tac-Toe-2?sort=year%2Casc&limit=500
6 / 251 	: 7 	 https://www.discogs.com/artist/1258225-Toe-Fat?sort=year%2Casc&limit=500
7 / 251 	: 5 	 https://www.discogs.com/artist/14882-Toe?sort=year%2Casc&limit=500
8 / 251 	: 6 	 https://www.discogs.com/artist/133959-Tic-Tac-Toe?sort=year%2Casc&limit=500
9 / 251 	: 6 	 https://www.discogs.com/artist/797565-Toe-2?sort=year%2Casc&limit=500
10 / 251 	: 7 	 https://www.discogs.com/artist/2078529-Don-Toe-Prodaehl?sort=year%2Casc&limit=500
11 / 251 	: 6 	 https://www.discogs.com/artist

None
<h1 class="hide_mobile">Regine Toelg</h1>
  --> This file is 19.8kB.
77 / 251 	: 7 	 https://www.discogs.com/artist/3139784-Toe-Blake?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toe Blake</h1>
  --> This file is 15.1kB.
78 / 251 	: 7 	 https://www.discogs.com/artist/1643286-Sugar-Toe-Monroe?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Sugar Toe Monroe</h1>
  --> This file is 15.6kB.
79 / 251 	: 7 	 https://www.discogs.com/artist/2573190-Ton-Toe?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Ton-Toe</h1>
  --> This file is 15.2kB.
80 / 251 	: 7 	 https://www.discogs.com/artist/2008474-Wolfgang-Toepffer?sort=year%2Casc&limit=500
81 / 251 	: 7 	 https://www.discogs.com/artist/2930290-Corn-Toe?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Corn Toe</h1>
  --> This file is 15.9kB.
82 / 251 	: 7 	 https://www.discogs.com/artist/5493654-Kaitlyn-Toepperwein?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Kaitlyn Toepperwein</h1>
  --> This file 

None
<h1 class="hide_mobile">Hannes Toegel</h1>
  --> This file is 15.7kB.
130 / 251 	: 7 	 https://www.discogs.com/artist/5225304-Toe-Cutter-2?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toe Cutter (2)</h1>
  --> This file is 15.0kB.
131 / 251 	: 7 	 https://www.discogs.com/artist/4729115-Nick-Toerner?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Nick Toerner</h1>
  --> This file is 15.1kB.
132 / 251 	: 7 	 https://www.discogs.com/artist/3154033-Kim-Yong-Toek?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Kim Yong Toek</h1>
  --> This file is 16.5kB.
133 / 251 	: 7 	 https://www.discogs.com/artist/1119571-Toern?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toern</h1>
  --> This file is 15.1kB.
134 / 251 	: 6 	 https://www.discogs.com/artist/234760-Toe-Cutter?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toe Cutter</h1>
  --> This file is 15.0kB.
135 / 251 	: 7 	 https://www.discogs.com/artist/1220816-Robert-Toegel?sort=year%2Casc&limit=500

180 / 251 	: 7 	 https://www.discogs.com/artist/3232788-Hein-Toerien?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Hein Toerien</h1>
  --> This file is 16.4kB.
181 / 251 	: 7 	 https://www.discogs.com/artist/2427706-The-Mumber-Toes?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">The Mumber Toes</h1>
  --> This file is 15.1kB.
182 / 251 	: 7 	 https://www.discogs.com/artist/1676876-Adolf-Toegel?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Adolf Toegel</h1>
  --> This file is 17.7kB.
183 / 251 	: 6 	 https://www.discogs.com/artist/613004-Toe-Jam-People?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toe Jam People</h1>
  --> This file is 15.4kB.
184 / 251 	: 7 	 https://www.discogs.com/artist/2331207-Toez-3?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Toez (3)</h1>
  --> This file is 17.2kB.
185 / 251 	: 7 	 https://www.discogs.com/artist/1432309-Armin-Toerkell?sort=year%2Casc&limit=500
None
<h1 class="hide_mobile">Armin Toerkell</h1>
  --> This

HTTPError: HTTP Error 404: NOT FOUND

In [ ]:
%load_ext autoreload
%autoreload

from collection import Counter

refCounts = Counter(disc.getCollectionRefCountsData())
artistIDs = disc.getArtistIDToRefData()

In [ ]:
errs = getFile("colErrs.json")
len(errs)

In [ ]:
n = len(refCounts)
nNext = len(errs)

nFalse = 0
nNone  = 0
nTrue  = 0

for i,(ref,cnts) in enumerate(refCounts.most_common()):
    if i % 2500 == 0:
        print("Processed {0}/{1}  (Errs={2})".format(i,n, len(errs)))
        
    if errs.get(ref) is not None:
        continue
        
    artistID = dutils.getArtistID(ref)
    if artistID is None:
        continue
    if artistIDs.get(artistID) is None:
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if isFile(savename):
            continue
            
        try:
            retval = arts.downloadArtistURL(url=url, savename=savename, parse=True, debug=True)
        except:
            retval = None
            
        if retval is None:
            nNone += 1
            errs[ref] = "None"
        if retval is False:
            nFalse += 1
            errs[ref] = "False"

    if (nNone % 100 == 0 or nFalse % 100 == 0) and (nNone > 0 and nFalse > 0):
        print("There have been {0} errors so far...".format(nNone))
        print("There have been {0} no saves so far...".format(nFalse))
        print("There have been {0} full saves so far...".format(nTrue))        
        saveFile(ifile="colErrs.json", idata=errs)


            
    if len(errs) == nNext + 250:        
        saveFile(ifile="colErrs.json", idata=errs)
        break

In [ ]:
len(errs)

In [ ]:
errs2 = getFile("colErrs.json")
len(errs2)

In [ ]:
276,364

   221,430

   253,670

   260,874

   291,460

   229,137


2019 Month

Previous Data

Recent Data

January

279975

276341

February

225664

221255

March

254895

253661

April

260299

260835

May

294924

291434

June

234720

229125

